## Project Dublin Bus 


## Change the data format, remove duplication and  save it 

## Import package and data 

In [1]:
#Import the required packages
#Import package pandas for data analysis
import pandas as pd

# Import package numpy for numeric computing
import numpy as np

# Import package matplotlib  for visualisation/plotting
import matplotlib.pyplot as plt

#import matplotlib.patches as mpatches
import seaborn as sns

#import package matplotlib for import plots to PDF
from matplotlib.backends.backend_pdf import PdfPages

#For showing plots directly in the notebook
#%matplotlib inline

# Reading from a csv file, into a data frame

import time 
begin_time=time.time()

In [2]:
#Change the columns's names 
Columns_name = ['Timestamp','Line_ID','Direction','Journey_Pattern_ID','Date','Vehicle_Journey_ID','Operator',
              'Congestion','Lon_WGS84','Lat_WGS84','Delay','Block_ID','Vehicle_ID','Stop_ID','At_Stop']

In [3]:
#This is the data of January 2013
df = pd.read_csv('siri.20121106.csv',names=Columns_name, dtype={ 'Journey_Pattern_ID': object} )
df.head(3)

/Users/April/anaconda/envs/comp47350/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Timestamp,Line_ID,Direction,Journey_Pattern_ID,Date,Vehicle_Journey_ID,Operator,Congestion,Lon_WGS84,Lat_WGS84,Delay,Block_ID,Vehicle_ID,Stop_ID,At_Stop
0,1352160000000000,15,0,00150001,2012-11-05,5826,RD,0,-6.258584,53.340099,-361,15013,33210,4870,0
1,1352160000000000,46,0,046A1002,2012-11-05,7267,D2,0,-6.259093,53.345425,-1101,46004,36024,794,0
2,1352160000000000,14,0,00140001,2012-11-05,6206,D2,0,-6.257329,53.287521,-126,14003,33325,1047,0


In [4]:
# Check how many rows we have after loaded the csv file 
df.shape

(1765912, 15)

#  Data Cleaning 


In [5]:
#Drop duplicate rows 
df=df.drop_duplicates()
#Drop constant column
#df.drop(['Direction','Operator'],axis=1,inplace=True)
#Drop Unused column
#df.drop(['Line_ID','Congestion','Block_ID','Vehicle_ID'],axis=1,inplace=True)

In [6]:
# Check how many rows we have after remove duplication
df.shape

(1765912, 15)

## Remove all Journey_Pattern_ID is null

In [7]:
# Drop all Journey_Pattern_ID is null 
df=df.drop(df.index[df['Journey_Pattern_ID'].isnull()])

In [8]:
#Check how many rows we deleted(Before it is (10856935, 15) )
df.shape

(1765848, 15)

In [9]:
# Drop all Journey_Pattern_ID is null 
df=df.drop(df.index[df['Journey_Pattern_ID'] == 'null'])

In [10]:
#Check how many rows we deleted(Before it is (10856935, 15) )
df.shape

(1495764, 15)

In [58]:
#Print all the unique Journey_Pattern_ID
#print("Unique Journey_Pattern_ID ", pd.unique(df.Journey_Pattern_ID.ravel()))

## Remove some noise data ( Like only have less than 45 rows in one trip  )
## Those data are not actually a full run 

In [59]:
df_groupby=df.groupby(['Vehicle_Journey_ID','Date','Journey_Pattern_ID'])

#size = df_groupby.size()
#size=size[size<13]

#keys=df_groupby.groups

# Filter all the data only have less than 13 rows data 
df_short_Journ=df_groupby.filter(lambda x: len(x) < 45 )

In [60]:
# Drop all the Journery that are less than a number 

df=pd.concat([df, df_short_Journ]).drop_duplicates(keep=False)

In [61]:
#Check how many rows we deleted
df.shape


(9160370, 15)

# Solution to check the long time trip issue
## Check the time gap of two gps coordinations, if there are big than one minute, delete the first one 

In [5]:
df['Timestamp'] = df['Timestamp'] //1000000
df.head(3)

,Timestamp,Line_ID,Direction,Journey_Pattern_ID,Date,Vehicle_Journey_ID,Operator,Congestion,Lon_WGS84,Lat_WGS84,Delay,Block_ID,Vehicle_ID,Stop_ID,At_Stop
0,1352160000,15,0,00150001,2012-11-05,5826,RD,0,-6.258584,53.340099,-361,15013,33210,4870,0
1,1352160000,46,0,046A1002,2012-11-05,7267,D2,0,-6.259093,53.345425,-1101,46004,36024,794,0
2,1352160000,14,0,00140001,2012-11-05,6206,D2,0,-6.257329,53.287521,-126,14003,33325,1047,0


In [12]:
df['Next_Timestamp']=df.groupby(['Vehicle_Journey_ID','Journey_Pattern_ID','Date'])['Timestamp'].shift(-1)

In [13]:
def get_rid_of_nulls(row):
    if pd.isnull(row['Next_Timestamp']):
        return row['Timestamp']
    else:
        return row['Next_Timestamp']

df['Next_Timestamp']=df.apply(get_rid_of_nulls, axis=1)

In [14]:
df.tail()

,Timestamp,Line_ID,Direction,Journey_Pattern_ID,Date,Vehicle_Journey_ID,Operator,Congestion,Lon_WGS84,Lat_WGS84,Delay,Block_ID,Vehicle_ID,Stop_ID,At_Stop,Next_Timestamp
1765907,1352246396,15,0,015A0001,2012-11-06,3130,RD,0,-6.322834,53.304768,-241,15009,33490,1104,1,1.352246e+09
1765908,1352246396,33,0,00331005,2012-11-06,269,SL,0,-6.125801,53.578407,18,33011,43052,3816,0,1.352246e+09
1765909,1352246396,56,0,056A0001,2012-11-06,2078,RD,0,-6.371067,53.306877,-764,56002,33471,2739,0,1.352246e+09
1765910,1352246396,66,0,066A0001,2012-11-06,2940,PO,0,-6.429667,53.355366,-121,66001,40008,3994,1,1.352246e+09
1765911,1352246396,43,0,00430001,2012-11-06,3924,CF,0,-6.213805,53.453159,-309,43002,38075,5084,0,1.352246e+09


In [15]:
df['Next_Timestamp'] = df.Next_Timestamp.astype(int)

In [16]:
df['Between_Time']=df['Next_Timestamp']-df['Timestamp']

In [17]:
df_long=df.loc[(df['Between_Time']>120)]

In [18]:
df_long.head(15)

,Timestamp,Line_ID,Direction,Journey_Pattern_ID,Date,Vehicle_Journey_ID,Operator,Congestion,Lon_WGS84,Lat_WGS84,Delay,Block_ID,Vehicle_ID,Stop_ID,At_Stop,Next_Timestamp,Between_Time
2576,1352160349,56,0,056A1001,2012-11-05,2063,RD,0,-6.237294,53.342300,-616,56001,33230,353,0,1352161087,738
4562,1352160728,39,0,039A0001,2012-11-05,3705,PO,0,-6.273350,53.352333,-731,39008,33595,7011,0,1352161108,380
6057,1352161108,39,0,039A0001,2012-11-05,3705,PO,0,-6.274783,53.353317,-731,39008,33595,7011,0,1352161287,179
6872,1352161386,39,0,039A0001,2012-11-05,3705,PO,0,-6.274783,53.353333,-731,39008,33595,7011,0,1352161528,142
7439,1352161627,39,0,039A0001,2012-11-05,3705,PO,0,-6.274766,53.353317,-731,39008,33595,7011,0,1352161748,121
8801,1352162737,13,0,00130008,2012-11-05,4094,HN,0,-6.276967,53.416916,-531,13103,33338,2166,0,1352163167,430
8933,1352163087,39,0,039A0001,2012-11-05,3705,PO,0,-6.274800,53.353333,-731,39008,33595,7011,0,1352163368,281
8956,1352163167,13,0,00130008,2012-11-05,4094,HN,0,-6.276967,53.416916,-531,13103,33338,2166,0,1352163586,419
8984,1352163302,79,0,00790001,2012-11-05,3511,CD,0,-6.296617,53.347698,29,79003,33191,2907,0,1352163730,428
9036,1352163586,13,0,00130008,2012-11-05,4094,HN,0,-6.276967,53.416916,-531,13103,33338,2166,0,1352169353,5767


In [19]:
df_long.shape

(1633, 17)

In [20]:
df=df.drop(df.index[df['Between_Time']>120])

In [21]:
# Drop the feature we don't need now 
df.drop(['Next_Timestamp','Between_Time'],axis=1,inplace=True)

In [22]:
#Check how many rows we deleted(Before it is (9211192, 15) )
df.shape

(1494131, 15)

# Add Distance feature 

In [41]:
#df_one=df.loc[df['Journey_Pattern_ID']=='041B0002']
#df_one.reset_index(drop=True, inplace=True)

In [45]:
#df_one.head(3)

,Timestamp,Line_ID,Direction,Journey_Pattern_ID,Date,Vehicle_Journey_ID,Operator,Congestion,Lon_WGS84,Lat_WGS84,Delay,Block_ID,Vehicle_ID,Stop_ID,At_Stop,Distance
0,1352201449000000,41,0,041B0002,2012-11-06,51,SL,0,-6.255460,53.349075,41,41008,33425,288,0,0
1,1352201469000000,41,0,041B0002,2012-11-06,51,SL,0,-6.254753,53.351208,-57,41008,33425,1171,0,242
2,1352201487000000,41,0,041B0002,2012-11-06,51,SL,0,-6.256154,53.353180,-126,41008,33425,1172,0,480


In [47]:
from geopy.distance import distance


        
def group_df(groupby_params, df):
    """
    :param groupby_params: list of feature names as strings by which to group by
    :param df: dataframe that is to be grouped
    :return: a pandas groupby object
    """
    if len(groupby_params) == 0:
        print("You did not provide any groupby parameters")
        return None
    else:
        df_grouped = df.groupby(groupby_params)
        return df_grouped

def coor():
    prev = yield
    running_dist = 0
    
    while True:
        next_ = yield running_dist
        one_dist=distance(prev, next_).meters
        if one_dist>1000:
            running_dist += 100
        else:
            running_dist += one_dist
        #print("The running_dist is ",running_dist)
        prev = next_
        
def add_distance_all_runs(df):
    """
    :param df: dataframe before distance has been added
    :return: dataframe with distances for each run
    """
    df['Distance'] = -1
    vj_groups = []
    df_grouped = group_df(['Journey_Pattern_ID','Date'], df)
    for journey_name, journey_group in df_grouped:
        vehicle_groups = group_df(['Vehicle_Journey_ID'], journey_group)
        for v_name, v_group in vehicle_groups:
            c = coor()
            next(c)
            #print("A new loop")
            for index, row in v_group.iterrows():
                dist = c.send((row['Lat_WGS84'], row['Lon_WGS84']))   
                v_group.set_value(index, 'Distance', dist)
            if v_group['Distance'].max() < 36000:
                vj_groups.append(v_group)

    return pd.concat(vj_groups)

df=add_distance_all_runs(df)

A new loop
The running_dist is  242.01142419948468
The running_dist is  480.48638121659064
The running_dist is  659.9375178773328
The running_dist is  737.8439421246511
The running_dist is  813.8084389980299
The running_dist is  929.7975772335211
The running_dist is  1060.5209245068943
The running_dist is  1174.0004659717006
The running_dist is  1246.003244089239
The running_dist is  1246.003244089239
The running_dist is  1356.7385757758577
The running_dist is  1395.4008087651032
The running_dist is  1679.52139792173
The running_dist is  1740.3474020531103
The running_dist is  1740.3474020531103
The running_dist is  1847.8709304989936
The running_dist is  2090.580454323054
The running_dist is  2103.727492652708
The running_dist is  2198.6635667834826
The running_dist is  2198.6635667834826
The running_dist is  2265.056219272132
The running_dist is  2432.3090746716766
The running_dist is  2580.3458586388942
The running_dist is  2600.7242411290886
The running_dist is  2709.2231669630155


# For some run, the trip is too long, then we could remove all the row after three and half hours. As some bus didn't stop after finish one trip 

In [ ]:
# Add new feature Trip_Time 
zscore=lambda x: (x - x.min())
df['Trip_Time']= df.groupby(['Vehicle_Journey_ID','Journey_Pattern_ID','Date'])['Timestamp'].transform(zscore)

In [ ]:
# Drop all the rows' Trip_Time is longer than four hours 
df=df.drop(df.index[df['Trip_Time']>=14400])

# Add some other features 

In [6]:
#Convert Timestamp to datetime and new column datetime to data frame 
df['datetime'] = pd.to_datetime(df['Timestamp'],unit='s')
df.head(5)

,Timestamp,Line_ID,Direction,Journey_Pattern_ID,Date,Vehicle_Journey_ID,Operator,Congestion,Lon_WGS84,Lat_WGS84,Delay,Block_ID,Vehicle_ID,Stop_ID,At_Stop,datetime
0,1352160000,15,0,00150001,2012-11-05,5826,RD,0,-6.258584,53.340099,-361,15013,33210,4870,0,2012-11-06 00:00:00
1,1352160000,46,0,046A1002,2012-11-05,7267,D2,0,-6.259093,53.345425,-1101,46004,36024,794,0,2012-11-06 00:00:00
2,1352160000,14,0,00140001,2012-11-05,6206,D2,0,-6.257329,53.287521,-126,14003,33325,1047,0,2012-11-06 00:00:00
3,1352160002,41,0,041B0002,2012-11-05,61,SL,0,-6.264167,53.453217,-623,41008,33631,3874,1,2012-11-06 00:00:02
4,1352160002,63,0,null,2012-11-05,1116,D2,0,-6.171050,53.259201,292,63003,33137,3283,0,2012-11-06 00:00:02


In [7]:
# Add hourOfDay to dataframe
df['datetime']=pd.to_datetime(df['datetime'])
df['HourOfDay']=df['datetime'].dt.hour

In [8]:
#ADD day of the week to the date frame 
df['day_of_week'] = df['datetime'].dt.weekday_name
df.head(3)

,Timestamp,Line_ID,Direction,Journey_Pattern_ID,Date,Vehicle_Journey_ID,Operator,Congestion,Lon_WGS84,Lat_WGS84,Delay,Block_ID,Vehicle_ID,Stop_ID,At_Stop,datetime,HourOfDay,day_of_week
0,1352160000,15,0,00150001,2012-11-05,5826,RD,0,-6.258584,53.340099,-361,15013,33210,4870,0,2012-11-06,0,Tuesday
1,1352160000,46,0,046A1002,2012-11-05,7267,D2,0,-6.259093,53.345425,-1101,46004,36024,794,0,2012-11-06,0,Tuesday
2,1352160000,14,0,00140001,2012-11-05,6206,D2,0,-6.257329,53.287521,-126,14003,33325,1047,0,2012-11-06,0,Tuesday


In [ ]:
# Add midweek feature 
df['midweek'] = df['day_of_week'].map({'Monday': 1, 'Tuesday': 0,'Wednesday':0,'Thursday':0,'Friday': 0,'Saturday':1,'Sunday':1
                                          })

In [ ]:
# Only keep the first row for every trip at the same Stop(keep both at or not_at stop)
df = df.drop_duplicates(['Vehicle_Journey_ID','Journey_Pattern_ID','Date','Stop_ID','At_Stop'])

# Add weather data 

In [9]:
def add_time_bin_column(x):
    if x <= 4:
        return 'early_am'
    elif x >= 5 and x <= 12:
        return 'am'
    elif x >= 12 and x <= 20:
        return 'pm'
    elif x >= 21:
        return 'late_pm'

df['time_bin']=df['HourOfDay'].map(lambda x:add_time_bin_column(x))
# Add midweek feature 

#s.map(lambda x: 'this is a string {}'.format(x),na_action='ignore')


In [10]:
df.head()

,Timestamp,Line_ID,Direction,Journey_Pattern_ID,Date,Vehicle_Journey_ID,Operator,Congestion,Lon_WGS84,Lat_WGS84,Delay,Block_ID,Vehicle_ID,Stop_ID,At_Stop,datetime,HourOfDay,day_of_week,time_bin
0,1352160000,15,0,00150001,2012-11-05,5826,RD,0,-6.258584,53.340099,-361,15013,33210,4870,0,2012-11-06 00:00:00,0,Tuesday,early_am
1,1352160000,46,0,046A1002,2012-11-05,7267,D2,0,-6.259093,53.345425,-1101,46004,36024,794,0,2012-11-06 00:00:00,0,Tuesday,early_am
2,1352160000,14,0,00140001,2012-11-05,6206,D2,0,-6.257329,53.287521,-126,14003,33325,1047,0,2012-11-06 00:00:00,0,Tuesday,early_am
3,1352160002,41,0,041B0002,2012-11-05,61,SL,0,-6.264167,53.453217,-623,41008,33631,3874,1,2012-11-06 00:00:02,0,Tuesday,early_am
4,1352160002,63,0,null,2012-11-05,1116,D2,0,-6.171050,53.259201,292,63003,33137,3283,0,2012-11-06 00:00:02,0,Tuesday,early_am


In [15]:
df_weather=pd.read_csv('weather_output.csv')

In [16]:
df_weather.head()

,Date,time_bin,cloud,rain,temp,wind
0,2012-11-01,early_am,3.60,0.00,1.30,7.00
1,2012-11-01,am,5.75,0.00,1.70,12.50
2,2012-11-01,pm,3.88,0.12,5.29,17.25
3,2012-11-01,late_pm,2.67,0.00,4.73,19.67
4,2012-11-02,early_am,6.00,0.00,4.66,16.80


In [17]:
df_weather.dtypes

Date         object
time_bin     object
cloud       float64
rain        float64
temp        float64
wind        float64
dtype: object

In [18]:
df.shape

(1765912, 19)

In [19]:
df = pd.merge(df, df_weather,  how='left', left_on=['Date','time_bin'], right_on = ['Date','time_bin'])

In [20]:
df.shape

(1765912, 23)

for index, row in df.iterrows():
    date = row['Date'][8:]

def add_weather_columns(df, weather_data):
    """
    :param df: a dataframe
    :param weather_data: a json file with weather data in the appropriate format
    :return: a dataframe with weather data added
    """
    df['wind'] = 0
    df['rain'] = 0
    df['cloud'] = 0
    df['temp'] = 0
    weather_options = ['wind', 'rain', 'cloud', 'temp']
    for index, row in df.iterrows():
        date = row['Date']
        timebin=row['time_bin']
        df_select=weather_data.loc[(weather_data['time_bin']==timebin)&(weather_data['date']==date)]
        row=df_weather.head(1)
        for opt in weather_options:
            value = row[opt]
            df.set_value(index, opt, value)

    return df
add_weather_columns(df, df_weather)

# Save the file for modeling 

In [27]:
#Save the data for next step
df.to_csv('dublin_2012_week1.csv',index=False)

In [ ]:
End_time=time.time()
Duringtime=End_time-begin_time
print("The time used is ",Duringtime//60)